In [2]:
import numpy as np
from tight_binding.hamiltonians import static_bloch_hamiltonian_kagome
from tight_binding.hamiltonians import driven_bloch_hamiltonian_kagome
from tight_binding.utilities import compute_reciprocal_lattice_vectors_2D
import matplotlib.pyplot as plt
from tight_binding.utilities import compute_time_evolution_operator
from matplotlib import cm
from tight_binding.solver import driven_bandstructure2D

In [14]:
a = np.linspace(0,10,21) * np.pi
a / np.pi

array([ 0. ,  0.5,  1. ,  1.5,  2. ,  2.5,  3. ,  3.5,  4. ,  4.5,  5. ,
        5.5,  6. ,  6.5,  7. ,  7.5,  8. ,  8.5,  9. ,  9.5, 10. ])

In [15]:
a = (a 
                            + 2*np.pi
                            *np.floor((0/np.pi - a/np.pi) / 2
                              + 1))
a / np.pi

array([2. , 0.5, 1. , 1.5, 2. , 0.5, 1. , 1.5, 2. , 0.5, 1. , 1.5, 2. ,
       0.5, 1. , 1.5, 2. , 0.5, 1. , 1.5, 2. ])

In [5]:
int(-0.5)

0

In [6]:
int(1.9)

1

In [7]:
int(-1.9)

-1

In [8]:
floor(-1.9)

NameError: name 'floor' is not defined